Dependencies

In [1]:
import numpy as np
import pandas as pd
import open3d as o3d
import transforms3d as tf3d
import csv
from pyntcloud import PyntCloud as pt

__Data Processing Steps__

In [2]:
def data_array_from_column(df, column_name):
    vectors = df[column_name].apply(eval)
    vectors_df = pd.DataFrame(vectors.tolist())
   

    return np.asarray(vectors_df)

Data Extractions

In [3]:
data = pd.read_csv("test4.csv")
childsizes_all = data.loc[data['PropertyName']=='childsizes']
childsizes_array = data_array_from_column(childsizes_all,'Data')
childpositions_all = data.loc[data['PropertyName']=='childpositions']
childpositions_array = data_array_from_column(childpositions_all,'Data')
childorientations_all = data.loc[data['PropertyName']=='childorientations']
childorientations_array = data_array_from_column(childorientations_all,'Data')



Convert from Radians to Degrees and find relative angles

In [5]:
converted_angles = []

for i in range(len(childorientations_array)):
    for j in range(len(childorientations_array[0])):
        for k in range(len(childorientations_array[0][0])):
            radian_values = (childorientations_array[i][j][k])*57.2957795
            converted_angles.append(radian_values)


converted_angles = np.reshape(converted_angles,(6,73,3))

In [6]:
only_roll = []

for i in range(len(converted_angles)):
    for j in range(len(converted_angles[0])):
        roll = converted_angles[i][j][0]
        only_roll.append(roll)

only_roll = np.reshape(only_roll,(6,73))

In [7]:
reference_value = [i[0]for i in only_roll]
allothervalues =  np.delete(only_roll,0,axis=1)
relative_values = []
for i in range(len(allothervalues)):
    rel_value = allothervalues[i] - reference_value[i]
    relative_values.append(rel_value)



Transformation Matrix


In [8]:
tf_data = pd.read_csv("tfdata.csv")
tf_array = data_array_from_column(tf_data,'Data')

Create Pose of the Objects

In [9]:
childPose = []
for i in range(len(childpositions_array)):
    for j in range(len(childpositions_array[0])):
        arr1= childpositions_array[i][j]
        arr2 = childorientations_array[i][j]
        temp_array = np.concatenate((arr1, arr2 ), axis=0)
        childPose.append(temp_array)

childPose = np.asarray(childPose)   
childPose = childPose.reshape(6,73,6)


Transform from World Frame to Vehicle Frame

In [10]:
new_worldframes = []
for i in range(len(tf_array)):
    temp_array = np.asarray(tf_array[i])
    temp_array = temp_array.reshape(4,4)
    new_worldframes.append(temp_array)

In [11]:
inverse_tf = []
for i in range(len(new_worldframes)):
    inv_tf = np.linalg.inv(new_worldframes[i])
    inverse_tf.append(inv_tf)

In [12]:
transformed_position =[]
for i in range(len(childPose)):
    for j in range(len(childPose[0])):
        positionA = np.reshape(childPose[i][j][:3],(3,1))
        RotAB = inverse_tf[i][:3,:3]
        translationAB = np.reshape(inverse_tf[i][:3,3],(3,1))
        alpha = np.dot(RotAB,positionA)
        beta = alpha + translationAB
        transformed_position.append(beta)


transformed_position = np.reshape(transformed_position,(6,73,3))

Create  bounding box edge coordinates

In [13]:
BB_length = []
BB_width = []
BB_height = []

for i in range(len(childsizes_array)):
    for j in range(len(childsizes_array[0])):
        length = (childsizes_array[i][j][0])/2
        width = (childsizes_array[i][j][1])/2
        height = (childsizes_array[i][j][2])/2
        
        BB_length.append(length)
        BB_width.append(width)
        BB_height.append(height)

BB_length = np.reshape(BB_length,(6,73))
BB_height = np.reshape(BB_height,(6,73))
BB_width = np.reshape(BB_width,(6,73))

In [14]:
BB_coordinates = []

for i in range(len(BB_height)):
    for j in range(len(BB_height[0])):
        x = transformed_position[i][j][0]
        y = transformed_position[i][j][1]
        z =  transformed_position[i][j][2]

        l = BB_length[i][j]
        h = BB_height[i][j]
        w = BB_width[i][j]

        edges = []
        e1 = (x-w,y-l,z-h)
        e2 = (x-w,y-l,z+h)
        e3 = (x-w,y+l,z+h)
        e4 = (x-w,y+l,z-h)
        e5 = (x+w,y-l,z-h)
        e6 = (x+w,y-l,z+h)
        e7 = (x+w,y+l,z+h)
        e8 = (x+w,y+l,z-h)
        edges.append(e1)
        edges.append(e2)
        edges.append(e3)
        edges.append(e4)
        edges.append(e5)
        edges.append(e6)
        edges.append(e7)
        edges.append(e8)

        BB_coordinates.append(edges)

BB_coordinates = np.reshape(BB_coordinates,(6,73,8,3))

Save as csv

In [15]:
fil_name = 'BB_coordinates'
with open(fil_name+'.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerows(BB_coordinates)

Testing one Object from timestep 0.2


In [16]:
Obj_BB_Coord = BB_coordinates[1][1]
loc = "C:\\Users\\Proteus\\Desktop\\RWTH\\HiWi\\UrbanTestDrive\\pcd\\test.pcd_simTime-199.99917603ms.pcd"
pc_read = o3d.io.read_point_cloud(loc)
pcd_data = np.asarray(pc_read.points)


Draw PCD

In [17]:
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pcd_data)
o3d.io.write_point_cloud("pcd1.ply",pcd1)

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(Obj_BB_Coord)
o3d.io.write_point_cloud("pcd2.ply",pcd2)

pcd1_read = o3d.io.read_point_cloud("pcd1.ply")
pcd2_read = o3d.io.read_point_cloud("pcd2.ply")

pcd1_read.paint_uniform_color([0,0,0.75]) # choose a unique rgb value for each point cloud
pcd2_read.paint_uniform_color([0,0.5,0])

bb_data = o3d.geometry.AxisAlignedBoundingBox.create_from_points(pcd2.points)
obb_data = o3d.geometry.AxisAlignedBoundingBox.get_oriented_bounding_box(bb_data)
# o3d.visualization.draw_geometries([pcd1_read,pcd2_read])
o3d.visualization.draw_geometries([pcd1_read,obb_data])